In [5]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row
from bokeh.models import CustomJS, Slider, Label, ColumnDataSource
output_notebook()

theta = 0 #input the value here
theta = np.radians(-theta)

#Inputs to be made text boxes
sig_x = 10

source = ColumnDataSource(data=dict(x=[1,1], y=[-1,1]))

fig = figure(
    title = 'Test of Text Rotation',
    plot_height = 300,
    plot_width = 300,
    x_range = (-3,3),
    y_range=(-3,3),
    toolbar_location = None
)
fig.line('x', 'y',source=source, line_width = 2)

citation = Label(x=1, y=1, text = str(10*sig_x),angle=theta)
fig.add_layout(citation)

amp_slider = Slider(start=0, end=360, value=theta, step=1, title="theta")

# Adding callback code
callback = CustomJS(args=dict(source=source ,val=amp_slider, lable=citation),
                code="""
const data = source.data;

var x = data['x'];
var y = data['y'];
var pi = Math.PI;
var theta = -1*(val.value) * (pi/180);
x[0]=(1*Math.cos(theta))-(1*Math.sin(theta));
x[1]=(1*Math.cos(theta))+(1*Math.sin(theta));
y[0]=(-1*Math.sin(theta))-(1*Math.cos(theta));
y[1]=(-1*Math.sin(theta))+(1*Math.cos(theta));
source.change.emit();

lable['x'] = x[1]
lable['y'] = y[1]
lable['angle'] = theta
lable.change.emit();
"""
)
amp_slider.js_on_change('value', callback)
layout = row(fig, amp_slider)
show(layout)

Loading BokehJS ...